# Runnable
- Prompt를 구성할 때, chain을 실행할 때 등 더 편의성과 효율성을 제공하기 위한 다양한 클래스가 구현되어 있다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [2]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    # max_new_tokens=8,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface.chat_models import huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is 10 times of {num}?\n")
chain = prompt | model
chain.invoke({"num": 10})

AIMessage(content='10 times 10 is 100.', id='run-bd9127a3-eef4-4d05-8315-b2e703784231-0')

# RunnablePassthrough
- 사용자의 입력을 전달하는 역할을 하며, 사용자가 map을 직접 넣지 않고 데이터만 넣을 수 있게 한다.
- `assign` 함수를 사용하면 입력 dict에 새로운 인수를 추가할 수 있다.

In [8]:
from langchain_core.runnables import RunnablePassthrough
runnable_chain = {"num": RunnablePassthrough()} | prompt | model
runnable_chain.invoke(10)

AIMessage(content='10 times 10 is 100.', id='run-0f017f07-2977-4fda-b677-37e24c9389cd-0')

In [9]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

In [12]:
# assign 함수를 통해 new_num이라는 새로운 인수를 추가한다.
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

`RunnablePassthrough`는 `RunnableParallel`과 같이 많이 사용된다.

In [13]:
from langchain_core.runnables import RunnableParallel
runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified = lambda x: x["num"] + 1,
)
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

# RunnableLambda
- 사용자 정의 함수를 chain에 추가할 수 있다.
- 대신 1개 인자를 받는 함수만 적용할 수 있다.
- 근데 RunnableLambda 없이도 함수를 chain에 넣으면 langchain에서 강제로 사용자 정의 함수를 Runnable로 변환하기 때문에, 함수를 바로 사용해도 가능하다.

In [16]:
from langchain_core.runnables import RunnableLambda
def extra(x):
    ext = x["extra"]
    return int(ext["num"]) * int(ext["mult"])

# (runnable | RunnableLambda(extra)).invoke({"num": 3})
(runnable | extra).invoke({"num": 3})

27